# L5: Automate Event Planning

In this lesson, you will learn more about Tasks.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Crew, Task

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key,get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [6]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [11]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [12]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [13]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [14]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [15]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [16]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [17]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [18]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
I need to find a venue in San Francisco that meets the criteria for a Tech Innovation Conference. I should consider the theme, size, and budget constraints of the event.

Action: Search the internet
Action Input: {"search_query": "venues for tech events in San Francisco"} 


Search results: Title: Tech conferences events in San Francisco, CA - Eventbrite
Link: https://www.eventbrite.com/d/ca--san-francisco/tech-conferences/
Snippet: Tech Networking Event by Startup Valley in San Francisco. Wed, Apr 30 • 7:00 PM. The Devil's Acre. Check ticket price on event. Save this event: Tech ...
---
Title: Networking Venues for Rent in San Francisco, CA - Tagvenue
Link: https://www.tagvenue.com/us/hire/networking-venues/san-francisco
Snippet: SoMa (South of Market): Known for its tech and startup scene

This is the agent final answer: Based on the search results and website content from Tagvenue, I have found several networking venues in San Francisco that could accommodate the Tech Innovation Conference. Some potential venues include:

1. Presidio Event & Office Space
- Location: Presidio, San Francisco, CA
- Description: Office Space in an Event Venue, private space
- Capacity: 100 guests
- Testimonial: "The venue was absolutely stunning. All the attendees complimented the area."
- Hourly hire fee: From $400

2. Edge & Node House of Web3
- Location: Presidio, San Francisco, CA
- Description: Bar, whole venue
- Capacity: 65 guests
- Description: A convivial neighborhood bar with a quirky, dressed-down tone that makes the space more relaxed and approachable.
- Hourly hire fee: From $50

3. The Newhall Nest
- Location: Financial District, San Francisco, CA
- Description: Rooftop at a Penthouse, private space
- Capacity: 70 guests
- Description: The roof deck offers a flexible layout pe

Action Input: {"search_query": "event venues in San Francisco for 500 participants capacity"} 


Search results: Title: Large Event Venues for Rent in San Francisco, CA - Tagvenue
Link: https://www.tagvenue.com/us/hire/large-event-venues/san-francisco
Snippet: We've got just the right venue for your occasion! Browse through our top large event venues and book your dream one in just a few clicks!
---
Title: The Most Fabulous Large Event Venues in San Francisco
Link: https://memo.thevendry.com/large-event-venues-san-francisco/
Snippet: 19 Fabulous Large Event Venues in San Francisco · 1. Storek · 2. The Archery · 3. Flamingo Resort & Spa · 4. SVN West · 5. Moscone Center · 6.
---
Title: The 25 Best Event Venues in San Francisco - Bizzabo
Link: https://www.bizzabo.com/blog/san-francisco-event-venues
Snippet: Featuring two levels, 50,000 square feet, and more, The Aquarium of the Bay in San Francisco offers a unique and engaging event venue. Located ...
---
Title: Spacious Large Event Venu

Exception in thread Thread-5 (_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/usr/local/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/site-packages/crewai/task.py", line 182, in _execute
    result = agent.execute_task(
             ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/crewai/agent.py", line 221, in execute_task
    result = self.agent_executor.invoke(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 163, in invoke
    raise e
  File "/usr/local/lib/python3.11/site-packages/langchain/chains/base.py", line 153, in invoke
    self._call(inputs, run_manager=run_manager)
  File "/us

- Display the generated `venue_details.json` file.

In [19]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'San Francisco',
 'booking_status': 'Available',
 'capacity': 100,
 'name': 'Presidio Event & Office Space'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [20]:
from IPython.display import Markdown
Markdown("marketing_report.md")

# Marketing Activities and Attendee Engagement Report

## Marketing Strategies for the Tech Innovation Conference

After conducting thorough research on potential marketing strategies for the Tech Innovation Conference to engage at least 500 participants, the following key strategies have been identified:

1. **Host a Giveaway:** Generating pre-event buzz with a giveaway can motivate potential attendees to register.

2. **Utilize Social Media:** Engage with potential attendees, share event updates, and promote speakers and sessions through various social media platforms.

3. **Leverage Email Marketing:** Utilize email marketing campaigns to reach out to potential attendees and provide them with relevant information about the conference.

4. **Partner With Industry Influencers:** Collaborate with industry influencers to help promote the conference and attract a larger audience.

5. **Offer Early Bird Discounts:** Encourage early registration by offering special discounts to early bird attendees.

## Expected Attendee Engagement

By implementing the above marketing strategies effectively, it is anticipated that the Tech Innovation Conference will be able to engage at least 500 potential attendees. The combination of engaging content, strategic promotion, and targeted outreach will help maximize event exposure and participation.

For further details on the Tech Innovation Conference and to stay updated on the latest event developments, please visit [Tech Innovation Conference Website](https://techinnovationconference.com).

Thank you for your attention and support in promoting this exciting event!